# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:
# Prepare data

from datasets import load_dataset, DatasetDict, concatenate_datasets

# this dataset has text labeleed according to its social appropriateness. 
dataset = load_dataset('allenai/prosocial-dialog')
combined_dataset = concatenate_datasets([dataset['train'], dataset['validation'], dataset['test']])
filtered_dataset = combined_dataset.filter(lambda example: example['safety_label'] in ['__casual__', '__needs_intervention__'])
df = filtered_dataset.to_pandas()

min_samples = min(df['safety_label'].value_counts()['__casual__'], df['safety_label'].value_counts()['__needs_intervention__'])
df_balanced = df.groupby('safety_label').apply(lambda x: x.sample(min_samples)).reset_index(drop=True)


/tmp/ipykernel_398/961986339.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby('safety_label').apply(lambda x: x.sample(min_samples)).reset_index(drop=True)


In [3]:
df_balanced.safety_label.value_counts()

safety_label
__casual__                21646
__needs_intervention__    21646
Name: count, dtype: int64

In [4]:
! pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [5]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

train_df, temp_df = train_test_split(df_balanced, test_size=0.4, stratify=df_balanced['safety_label'])
validation_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['safety_label'])
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)


In [6]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" 

def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['context'], truncation=True, padding='max_length', max_length=512)
    tokenized_inputs['labels'] = [0 if label == '__casual__' else 1 for label in examples['safety_label']]
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
validation_dataset = validation_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
validation_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})


Map:   0%|          | 0/25975 [00:00<?, ? examples/s]

Map:   0%|          | 0/8658 [00:00<?, ? examples/s]

Map:   0%|          | 0/8659 [00:00<?, ? examples/s]

In [7]:
from transformers import Trainer, TrainingArguments, GPT2ForSequenceClassification
from sklearn.metrics import accuracy_score
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np

# Load the model
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id
model.to(device)

# Configure and apply LoRA
config = LoraConfig(task_type=TaskType.SEQ_CLS,
                    r=8, 
                    lora_alpha=16,
                    lora_dropout=0.1,
                    #target_modules=["c_attn", "c_proj"]
                   )
lora_model = get_peft_model(model, config)
lora_model.to(device)

# Define compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    learning_rate=1e-5
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [8]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer_test = Trainer(
    model=lora_model,
    args=training_args,
    eval_dataset=dataset_dict['test'],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

lora_model.eval()
eval_results_before = trainer_test.evaluate()

In [9]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info(f"Evaluation results before training: {eval_results_before}")

INFO:__main__:Evaluation results before training: {'eval_loss': 2.12790846824646, 'eval_accuracy': 0.49994225661161795, 'eval_runtime': 323.1704, 'eval_samples_per_second': 26.794, 'eval_steps_per_second': 6.699}


Note the initial test set eval_accuracy': 0.49994225661161795, which makes sense given that the model hasna't been trained on this task yet.

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [10]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    learning_rate=1e-5
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['validation'],
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer)
)

trainer.train()

# Save the fine-tuned model and tokenizer
lora_model.save_pretrained('./finetuned')
tokenizer.save_pretrained('./finetuned')


Epoch,Training Loss,Validation Loss,Accuracy
1,0.683300,0.669151,0.596096
2,0.632700,0.642985,0.640333


('./finetuned/tokenizer_config.json',
 './finetuned/special_tokens_map.json',
 './finetuned/vocab.json',
 './finetuned/merges.txt',
 './finetuned/added_tokens.json')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [11]:
# After training
from peft import AutoPeftModelForSequenceClassification
inf_model = AutoPeftModelForSequenceClassification.from_pretrained("./finetuned", num_labels=2)
inf_model.config.pad_token_id = inf_model.config.eos_token_id

trainer_test = Trainer(
    model=inf_model,
    args=training_args,
    eval_dataset=dataset_dict['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

eval_results_after = trainer_test.evaluate()

logger.info(f"Evaluation after training: {eval_results_after}")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


INFO:__main__:Evaluation after training: {'eval_loss': 0.6388337016105652, 'eval_accuracy': 0.6470724102090311, 'eval_runtime': 325.3416, 'eval_samples_per_second': 26.615, 'eval_steps_per_second': 6.655}


Note that after training just 2 epochs, the test set performance on the test set has increased to eval_accuracy':  0.6470724102090311.

In [12]:
def classify(text: str):
    lookup = {0: '__casual__', 1: '__needs_intervention__'}
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inf_model.to(device)
    ins = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = inf_model(**ins)
        logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=1)
    cls_id = probabilities.argmax().item()
    pred_label = lookup[cls_id]
    return pred_label

# Examples
text = "The test is complete"
cls = classify(text)
print(f"Text: '{text}'\prediction: {cls}")

text = "I want to hurt you"
cls = classify(text)
print(f"Text: '{text}'\prediction: {cls}")

text = "Please hand me a pencil"
cls = classify(text)
print(f"Text: '{text}'\prediction: {cls}")

text = "Please hand me a pencil so that I may kill myself"
cls = classify(text)
print(f"Text: '{text}'\prediction: {cls}")

Text: 'The test is complete'\prediction: __casual__
Text: 'I want to hurt you'\prediction: __casual__
Text: 'Please hand me a pencil'\prediction: __casual__
Text: 'Please hand me a pencil so that I may kill myself'\prediction: __needs_intervention__


The model appears to works somewhat, and is improved realtive to the base gpt-2 model. It could likely be trained more for better results.